In [1]:
import os
import re
import inflect
import numpy as np

In [2]:
textos = [
    "Historia de Madrid: Madrid, la capital de España, tiene una rica historia que se remonta al siglo IX. La ciudad fue fundada por los musulmanes, y su nombre proviene del árabe “Majrit” que significa “fuente de agua”. A lo largo de los siglos, Madrid se ha convertido en un centro político, económico y cultural de España.",
    "El Palacio Real: El Palacio Real de Madrid es una de las residencias oficiales de la familia real española, aunque ahora se usa principalmente para ceremonias de estado. Con más de 3000 habitaciones, es el palacio real más grande de Europa Occidental y un testimonio del esplendor y la historia de España.",
    "Museo del Prado: El Museo del Prado es uno de los museos más famosos del mundo y un destino imprescindible para los amantes del arte que visitan Madrid. Alberga una impresionante colección de obras de Velázquez, Goya, El Greco y otros maestros europeos.",
    "Parque del Retiro: El Parque del Retiro es el pulmón verde de Madrid. Antiguamente perteneciente a la monarquía española, este parque de 118 hectáreas ofrece un respiro de la vida urbana con sus estanques, jardines y el Palacio de Cristal.",
    "Gastronomía Madrileña: Madrid es famosa por su vibrante escena gastronómica que incluye desde tapas en pequeños bares hasta restaurantes de alta cocina. Platos típicos como el cocido madrileño y las patatas bravas son esenciales para cualquier visitante.",
    "Gran Vía: La Gran Vía es una de las calles más famosas de Madrid, conocida por sus tiendas, teatros y arquitectura emblemática. Este bullicioso bulevar es un punto de encuentro para turistas y locales por igual.",
    "Real Madrid y el Estadio Santiago Bernabéu: El Real Madrid, uno de los clubes de fútbol más famosos y exitosos del mundo, juega sus partidos en el Estadio Santiago Bernabéu. Este estadio es una meca para los aficionados al fútbol y ofrece tours que permiten a los visitantes explorar su rica historia.",
    "Navidad en Madrid: Durante la temporada navideña, Madrid se transforma con mercados festivos, luces deslumbrantes y decoraciones. La Plaza Mayor se convierte en un enorme mercado de Navidad, donde se pueden encontrar todo tipo de regalos y delicias tradicionales.",
    "Nochevieja en Puerta del Sol: La Puerta del Sol es el corazón de Madrid y el lugar donde miles se reúnen cada Nochevieja para dar la bienvenida al año nuevo. Comer las doce uvas de la suerte al son de las campanadas del reloj es una tradición que todos deberían experimentar.",
    "Madrid Río: Madrid Río es un parque lineal a lo largo del río Manzanares y un ejemplo perfecto de cómo la ciudad ha transformado áreas industriales en espacios verdes vibrantes. Es un lugar popular para caminar, andar en bicicleta y disfrutar de actividades al aire libre.",
    "Historia de Dortmund: Dortmund, situada en la región del Ruhr en Alemania, tiene una historia que se remonta al año 882. Originalmente una pequeña villa, Dortmund creció para convertirse en una importante ciudad industrial, especialmente conocida por su producción de acero y cerveza.",
    "Westfalenpark: Westfalenpark es uno de los parques urbanos más grandes de Europa y un lugar destacado para los visitantes en Dortmund. El parque es famoso por su Rosarium, que alberga una de las colecciones más extensas de rosas en el mundo, y la torre Florian, que ofrece vistas panorámicas de la ciudad.",
    "Museo del Fútbol Alemán: Inaugurado en 2015, el Museo del Fútbol Alemán ofrece una experiencia interactiva que celebra la rica historia del fútbol alemán. Ubicado en Dortmund, el museo atrae a aficionados de todo el mundo y destaca por sus exposiciones que cuentan desde los primeros días del deporte hasta los triunfos en Copas del Mundo.",
    "Mercado de Navidad de Dortmund: El Mercado de Navidad de Dortmund es uno de los más grandes y más visitados de Alemania. Con su gigantesco árbol de Navidad, posiblemente el más alto de los mercados navideños del país, y más de 300 puestos que venden artesanías, alimentos y bebidas tradicionales, es un evento imperdible durante la temporada festiva.",
    "U-Tower - Centro de Arte y Creatividad: El U-Tower, una antigua cervecería reconvertida, es ahora un centro dinámico de arte y creatividad. Alberga el Museo Ostwall, espacios de exposiciones y estudios de artistas, y es un símbolo de la transformación cultural de Dortmund.",
    "Borussia Dortmund y el Signal Iduna Park: Borussia Dortmund es uno de los clubes de fútbol más populares y exitosos de Alemania. El equipo juega sus partidos en casa en el Signal Iduna Park, conocido por su increíble atmósfera y el famoso “Muro Amarillo”, la tribuna sur que alberga a 25000 aficionados apasionados.",
    "La cervecería Dortmund: Dortmund tiene una larga tradición cervecera, y la visita a una de sus muchas cervecerías es una parada obligatoria. Las cervecerías ofrecen tours que incluyen degustaciones de cervezas locales como Dortmunder Export, una lager que es un orgullo de la ciudad.",
    "La Reinoldikirche: La iglesia de San Reinold es el edificio más antiguo de Dortmund, datando del siglo XIII. Es un punto de referencia histórico importante y ofrece a los visitantes una mirada al pasado medieval de la ciudad.",
    "Dortmund durante la Segunda Guerra Mundial: Dortmund fue un objetivo significativo durante los bombardeos de la Segunda Guerra Mundial debido a su industria pesada. Hoy en día, varios monumentos y museos ofrecen reflexiones sobre este período y sus efectos en la ciudad.",
    "Zoológico de Dortmund: El Zoológico de Dortmund es un lugar excelente para familias y amantes de la naturaleza. Con más de 1500 animales y un enfoque especial en la con servación de especies amenazadas, el zoológico proporciona una experiencia educativa y entretenida."
]

In [3]:
categoria = ["Turismo", "Historia", "Guerra", "Fútbol", "Realeza", "Gastronomía", "Arte y Cultura", "Naturaleza y Medio Ambiente",
            "Arquitectura", "Comercio", "Festividades", "Transporte", "Educación", "Tecnología", "Deportes (otros)", "Religión",
            "Ocio y Entretenimiento", "Salud y Bienestar", "Industria", "Urbanismo"]

In [4]:
def clasificar_palabra(palabra):
    # Definir listas de palabras clave para cada categoría
    categorias = {
        "Turismo": ["playa", "hotel", "museo", "monumento", "excursión", "turista", "vacaciones", "turismo"],
        "Historia": ["revolución", "imperio", "siglo", "colonización", "rey", "batalla", "independencia", "historia"],
        "Guerra": ["soldado", "armamento", "batalla", "guerra", "conflicto", "ejército", "frente", "guerra"],
        "Fútbol": ["equipo", "estadio", "gol", "jugador", "partido", "campeonato", "liga", "fútbol"],
        "Realeza": ["rey", "reina", "palacio", "corona", "princesa", "monarquía", "duque", "realeza"],
        "Gastronomía": ["comida", "plato", "restaurante", "chef", "cocina", "sabor", "receta", "gastronomía"],
        "Arte y Cultura": ["museo", "exposición", "pintura", "escultura", "teatro", "danza", "música", "arte", "cultura"],
        "Naturaleza y Medio Ambiente": ["parque", "bosque", "río", "montaña", "flora", "fauna", "conservación", "naturaleza", "ambiente"],
        "Arquitectura": ["edificio", "diseño", "estructura", "monumento", "catedral", "rascacielos", "puente", "arquitectura"],
        "Comercio": ["mercado", "tienda", "venta", "compra", "negocio", "economía", "transacción", "comercio"],
        "Festividades": ["celebración", "festival", "evento", "feria", "navidad", "cumpleaños", "aniversario", "festividades"],
        "Transporte": ["coche", "avión", "tren", "autobús", "puerto", "carretera", "metro", "transporte"],
        "Educación": ["escuela", "universidad", "clase", "profesor", "estudiante", "aprendizaje", "examen", "educación"],
        "Tecnología": ["computadora", "internet", "software", "hardware", "innovación", "gadgets", "redes", "tecnología"],
        "Deportes (otros)": ["tenis", "baloncesto", "natación", "ciclismo", "atletismo", "gimnasia", "voleibol", "deportes"],
        "Religión": ["iglesia", "templo", "dios", "ritual", "sacerdote", "fe", "oración", "religión"],
        "Ocio y Entretenimiento": ["cine", "televisión", "juego", "videojuego", "parque de atracciones", "discoteca", "concierto", "ocio", "entretenimiento"],
        "Salud y Bienestar": ["hospital", "medicina", "doctor", "enfermera", "terapia", "salud", "bienestar"],
        "Industria": ["fábrica", "producción", "manufactura", "industrial", "sector", "empresa", "maquinaria", "industria"],
        "Urbanismo": ["ciudad", "planificación", "desarrollo", "infraestructura", "barrio", "urbanización", "espacio público", "urbanismo"]
    }

    # Convertir la palabra ingresada a minúsculas
    palabra = palabra.lower()
    
    # Verificar en cuál lista se encuentra la palabra
    for categoria, palabras_clave in categorias.items():
        if palabra in palabras_clave:
            return categoria
    
    return None


In [5]:
def crear_matriz(textos, categorias):
    matriz = [[0 for i in range(20)] for i in range(20)]
    
    inflect_engine = inflect.engine()
    
    for indice, texto in enumerate(textos):
        print(f"Texto N°: {indice + 1}")
        
        contenido = texto.lower()
        
        #palabras = re.findall(r'\b[a-zA-Z]+\b', contenido)
        # Dividir el contenido en palabras y eliminar caracteres no deseados
        eliminar = str.maketrans('', '', '.!?\'“”,:;_"[]{}‘’()')
        fila_palabras_temporal = [palabra.translate(eliminar) for palabra in contenido.split()]
        print(fila_palabras_temporal)
        #No sacamos plurales dado que se distorcionan las palabras en idioma español y dificulta la busqueda
        #Para eso mejor completamos las palabras plurales en el diccionario de categorias
        
        for palabra in fila_palabras_temporal:
            categoria_resultado = clasificar_palabra(palabra)
            if categoria_resultado != None:
                categoria_index = categorias.index(categoria_resultado)
                matriz[categoria_index][indice] += 1
                
    return matriz

In [6]:
matriz = crear_matriz(textos, categoria)

Texto N°: 1
['historia', 'de', 'madrid', 'madrid', 'la', 'capital', 'de', 'españa', 'tiene', 'una', 'rica', 'historia', 'que', 'se', 'remonta', 'al', 'siglo', 'ix', 'la', 'ciudad', 'fue', 'fundada', 'por', 'los', 'musulmanes', 'y', 'su', 'nombre', 'proviene', 'del', 'árabe', 'majrit', 'que', 'significa', 'fuente', 'de', 'agua', 'a', 'lo', 'largo', 'de', 'los', 'siglos', 'madrid', 'se', 'ha', 'convertido', 'en', 'un', 'centro', 'político', 'económico', 'y', 'cultural', 'de', 'españa']
Texto N°: 2
['el', 'palacio', 'real', 'el', 'palacio', 'real', 'de', 'madrid', 'es', 'una', 'de', 'las', 'residencias', 'oficiales', 'de', 'la', 'familia', 'real', 'española', 'aunque', 'ahora', 'se', 'usa', 'principalmente', 'para', 'ceremonias', 'de', 'estado', 'con', 'más', 'de', '3000', 'habitaciones', 'es', 'el', 'palacio', 'real', 'más', 'grande', 'de', 'europa', 'occidental', 'y', 'un', 'testimonio', 'del', 'esplendor', 'y', 'la', 'historia', 'de', 'españa']
Texto N°: 3
['museo', 'del', 'prado', 'el

In [7]:
for fila in matriz:
    print(fila)

[0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0]
[3, 1, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
[0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0]
[0, 3, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0]
[0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
queries = [
    "Historia medieval de las ciudades europeas",
    "Principales destinos turísticos en Europa",
    "Influencia de la realeza en la cultura europea",
    "Importancia de los parques urbanos en las ciudades",
    "Gastronomía típica en capitales europeas",
    "Eventos deportivos icónicos en Europa",
    "Celebraciones de Navidad en ciudades europeas",
    "Museos de arte importantes en Europa",
    "Efectos de la Segunda Guerra Mundial en ciudades europeas",
    "Arquitectura histórica en ciudades europeas",
    "Clubes de fútbol famosos y sus estadios en Europa",
    "Transformación urbana y regeneración de espacios",
    "Tradición cervecera en ciudades europeas",
    "Mercados y comercio tradicional en Europa",
    "Centros de arte y creatividad en ciudades modernas",
    "Actividades de ocio y entretenimiento en ciudades metropolitanas",
    "Conservación de la naturaleza y la vida silvestre en zonas urbanas",
    "Planificación de eventos culturales y festivales en ciudades",
    "Desarrollo del transporte y la infraestructura urbana",
    "Impacto de la tecnología en la vida urbana"
]

In [9]:
matriz_query = crear_matriz(queries, categoria)

Texto N°: 1
['historia', 'medieval', 'de', 'las', 'ciudades', 'europeas']
Texto N°: 2
['principales', 'destinos', 'turísticos', 'en', 'europa']
Texto N°: 3
['influencia', 'de', 'la', 'realeza', 'en', 'la', 'cultura', 'europea']
Texto N°: 4
['importancia', 'de', 'los', 'parques', 'urbanos', 'en', 'las', 'ciudades']
Texto N°: 5
['gastronomía', 'típica', 'en', 'capitales', 'europeas']
Texto N°: 6
['eventos', 'deportivos', 'icónicos', 'en', 'europa']
Texto N°: 7
['celebraciones', 'de', 'navidad', 'en', 'ciudades', 'europeas']
Texto N°: 8
['museos', 'de', 'arte', 'importantes', 'en', 'europa']
Texto N°: 9
['efectos', 'de', 'la', 'segunda', 'guerra', 'mundial', 'en', 'ciudades', 'europeas']
Texto N°: 10
['arquitectura', 'histórica', 'en', 'ciudades', 'europeas']
Texto N°: 11
['clubes', 'de', 'fútbol', 'famosos', 'y', 'sus', 'estadios', 'en', 'europa']
Texto N°: 12
['transformación', 'urbana', 'y', 'regeneración', 'de', 'espacios']
Texto N°: 13
['tradición', 'cervecera', 'en', 'ciudades', 'eu

In [10]:
for fila in matriz_query:
    print(fila)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
# Calcular la distancia coseno
def cosine_distance(vector1, vector2):
    dot_product = np.dot(vector1, vector2) # Calcular el producto punto con numpy
    norm_vector1 = np.linalg.norm(vector1) # Calcular la norma (raíz cuadrada de la suma de los cuadrados de los elementos)
    norm_vector2 = np.linalg.norm(vector2) # la norma es calculada con numpy, sin ello sería más código
    
    # Verificar si alguna de las normas es cero
    if norm_vector1 == 0 or norm_vector2 == 0:
        # Si una de las normas es cero, la distancia coseno será NaN
        return np.nan # Retornar NaN con numpy
    
    distance = dot_product / (norm_vector1 * norm_vector2) #Se calcula la distancia coseno por la formula
    return distance

In [12]:
distancias_resultados = []

for i, query_vector in enumerate(matriz_query):
    distances = []
    for row_vector in matriz:
        distance = cosine_distance(query_vector, row_vector)
        distances.append(distance)
    distancias_resultados.append(distances)
    print(f"Distancias coseno {i+1}:", distances)

Distancias coseno 1: [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
Distancias coseno 2: [0.0, 0.7276068751089989, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.3779644730092272]
Distancias coseno 3: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0]
Distancias coseno 4: [0.0, 0.48507125007266594, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.8944271909999159, 0.3779644730092272]
Distancias coseno 5: [0.5345224838248488, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4472135954999579, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0]
Distancias coseno 6: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0]
Distancias coseno 7: [0.46291004988627577, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7745966692414834, 0.0, 0.0, 0.2581988897471611, 0.2581988897471611, nan, nan, nan, nan, 0.0,

In [13]:
print(distancias_resultados)

[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], [0.0, 0.7276068751089989, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.3779644730092272], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0], [0.0, 0.48507125007266594, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.8944271909999159, 0.3779644730092272], [0.5345224838248488, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4472135954999579, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0], [0.46291004988627577, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7745966692414834, 0.0, 0.0, 0.2581988897471611, 0.2581988897471611, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.3779644730092272], [

In [36]:
distancias_con_indices = []

for i, distancias in enumerate(distancias_resultados):
    # Iterar sobre cada distancia en la lista de distancias
    for j, distancia in enumerate(distancias):
        # Si la distancia no es un NaN y no es un 0, agregarla junto con su índice a la lista de distancias con índices
        if not np.isnan(distancia) and distancia != 0.0:
            distancias_con_indices.append((distancia, (i, j))) # i es la query y j es el texto con distancia

In [37]:
print(distancias_con_indices)

[(0.7276068751089989, (1, 1)), (0.3779644730092272, (1, 19)), (0.48507125007266594, (3, 1)), (0.8944271909999159, (3, 18)), (0.3779644730092272, (3, 19)), (0.5345224838248488, (4, 0)), (0.4472135954999579, (4, 6)), (1.0, (5, 5)), (0.46291004988627577, (6, 0)), (0.7745966692414834, (6, 6)), (0.2581988897471611, (6, 9)), (0.2581988897471611, (6, 10)), (0.3779644730092272, (7, 19)), (0.769800358919501, (8, 7)), (0.3779644730092272, (8, 19)), (0.8944271909999159, (9, 9)), (0.8944271909999159, (9, 10)), (0.24253562503633297, (10, 1)), (0.8111071056538127, (10, 3)), (1.0, (11, 2)), (0.4472135954999579, (11, 18)), (0.3779644730092272, (11, 19)), (0.19245008972987526, (13, 7)), (0.3244428422615251, (16, 3)), (0.10846522890932808, (19, 1)), (0.8944271909999159, (19, 2)), (0.31622776601683794, (19, 8)), (0.4472135954999579, (19, 15)), (0.3999999999999999, (19, 18)), (0.5070925528371099, (19, 19))]


In [30]:
# Ordenar la lista de distancias con índices de menor a mayor distancia
distancias_ordenadas = sorted(distancias_con_indices)

In [32]:
print(distancias_ordenadas)

[(0.10846522890932808, (19, 1)), (0.19245008972987526, (13, 7)), (0.24253562503633297, (10, 1)), (0.2581988897471611, (6, 9)), (0.2581988897471611, (6, 10)), (0.31622776601683794, (19, 8)), (0.3244428422615251, (16, 3)), (0.3779644730092272, (1, 19)), (0.3779644730092272, (3, 19)), (0.3779644730092272, (7, 19)), (0.3779644730092272, (8, 19)), (0.3779644730092272, (11, 19)), (0.3999999999999999, (19, 18)), (0.4472135954999579, (4, 6)), (0.4472135954999579, (11, 18)), (0.4472135954999579, (19, 15)), (0.46291004988627577, (6, 0)), (0.48507125007266594, (3, 1)), (0.5070925528371099, (19, 19)), (0.5345224838248488, (4, 0)), (0.7276068751089989, (1, 1)), (0.769800358919501, (8, 7)), (0.7745966692414834, (6, 6)), (0.8111071056538127, (10, 3)), (0.8944271909999159, (3, 18)), (0.8944271909999159, (9, 9)), (0.8944271909999159, (9, 10)), (0.8944271909999159, (19, 2)), (1.0, (5, 5)), (1.0, (11, 2))]


In [43]:
# Ordenar la lista basada en el primer elemento de la tupla (i, j)
resultado_consulta = sorted(distancias_ordenadas, key=lambda x: x[1][0]) #Tomo segundo termino, y luego el primero

In [44]:
print(resultado_consulta)

[(0.3779644730092272, (1, 19)), (0.7276068751089989, (1, 1)), (0.3779644730092272, (3, 19)), (0.48507125007266594, (3, 1)), (0.8944271909999159, (3, 18)), (0.4472135954999579, (4, 6)), (0.5345224838248488, (4, 0)), (1.0, (5, 5)), (0.2581988897471611, (6, 9)), (0.2581988897471611, (6, 10)), (0.46291004988627577, (6, 0)), (0.7745966692414834, (6, 6)), (0.3779644730092272, (7, 19)), (0.3779644730092272, (8, 19)), (0.769800358919501, (8, 7)), (0.8944271909999159, (9, 9)), (0.8944271909999159, (9, 10)), (0.24253562503633297, (10, 1)), (0.8111071056538127, (10, 3)), (0.3779644730092272, (11, 19)), (0.4472135954999579, (11, 18)), (1.0, (11, 2)), (0.19245008972987526, (13, 7)), (0.3244428422615251, (16, 3)), (0.10846522890932808, (19, 1)), (0.31622776601683794, (19, 8)), (0.3999999999999999, (19, 18)), (0.4472135954999579, (19, 15)), (0.5070925528371099, (19, 19)), (0.8944271909999159, (19, 2))]


In [59]:
current_i = None

print("\nRESULTADOS DE BUSQUEDA \n")

for distancia, (i, j) in resultado_consulta:
    if i != current_i:
        if current_i is not None:
            print()  # Imprimir una línea en blanco para separar grupos
        current_i = i
        print(f"CONSULTA {i+1}: {queries[i]}")
        print()
    print(f"TEXTO {j+1}: {textos[j]}")
    print(f"Distancia hacia el texto {j+1}: {distancia}") #({distancia}, ({i}, {j}))
    print()


RESULTADOS DE BUSQUEDA 

CONSULTA 2: Principales destinos turísticos en Europa

TEXTO 20: Zoológico de Dortmund: El Zoológico de Dortmund es un lugar excelente para familias y amantes de la naturaleza. Con más de 1500 animales y un enfoque especial en la con servación de especies amenazadas, el zoológico proporciona una experiencia educativa y entretenida.
Distancia hacia el texto 20: 0.3779644730092272

TEXTO 2: El Palacio Real: El Palacio Real de Madrid es una de las residencias oficiales de la familia real española, aunque ahora se usa principalmente para ceremonias de estado. Con más de 3000 habitaciones, es el palacio real más grande de Europa Occidental y un testimonio del esplendor y la historia de España.
Distancia hacia el texto 2: 0.7276068751089989


CONSULTA 4: Importancia de los parques urbanos en las ciudades

TEXTO 20: Zoológico de Dortmund: El Zoológico de Dortmund es un lugar excelente para familias y amantes de la naturaleza. Con más de 1500 animales y un enfoque esp